### So you have your properties output ready and you need SHAPES?
Here's the very simple output dataframe we get to at the end of the BBC-Dataframes.ipynb

In [54]:
import numpy as np
import pandas as pd
out_properties = pd.read_csv('BBC_properties.csv')

In [55]:
out_properties

,properties.name,properties.article,properties.headline,properties.color
0,Italy,<p> At the First Breath of Wind : 1 vote</p><p...,1 critic,#000066
1,Mexico,"<p> 4 Months, 3 Weeks & 2 Days : 1 vote</p><p>...",1 critic,#000066
2,UK,"<p> 4 Months, 3 Weeks & 2 Days : 1 vote</p><p>...",1 critic,#000066
3,US,<p> Certified Copy : 2 votes</p><p> Eternal S...,4 critic,#000066


### Time for shapes

It doesn't have groups, which is fine--but it does have the properties 'name', 'article', 'headline', and 'color'. Notice that the 'name' property is a list of countries. And we have one row per country. If you've gotten this far you are ready to get some geometry--specifically, shapefiles.

An easy place to go and get shapefiles for countries is here:

https://geojson-maps.ash.ms/

(If you need files for some other level of information you will need to do some googling, and likely process a shapefile through mapshaper.)

But for countries, you can just go to the link above, click on the countries you want--or select the continents you want, and you'll get a geojson file with shape geometries for every country in the continents you selected.

I went and clicked on Italy, Mexico, UK, and US. And then downloaded the custom file which they named: 

`custom.geo.json`

Now it is time to bring in that file into pandas (just like we did with the courts in the leaflet tutorial).

In [56]:
import json
from pandas.io.json import json_normalize


In [57]:
with open('custom.geo.json') as json_data:
    geometry_data = json.load(json_data)


In [58]:
out_geometry = pd.DataFrame.from_dict(json_normalize(geometry_data['features']), orient='columns')
out_geometry

,geometry.coordinates,geometry.type,properties.abbrev,properties.abbrev_len,properties.adm0_a3,properties.adm0_a3_is,properties.adm0_a3_un,properties.adm0_a3_us,properties.adm0_a3_wb,properties.adm0_dif,...,properties.subregion,properties.subunit,properties.tiny,properties.type,properties.un_a3,properties.wb_a2,properties.wb_a3,properties.wikipedia,properties.woe_id,type
0,"[[[-97.14000830767071, 25.869997463478395], [-...",Polygon,Mex.,4,MEX,MEX,-99,MEX,-99,0,...,Central America,Mexico,-99,Sovereign country,484,MX,MEX,-99,-99,Feature
1,"[[[[15.520376010813834, 38.23115509699147], [1...",MultiPolygon,Italy,5,ITA,ITA,-99,ITA,-99,0,...,Southern Europe,Italy,-99,Sovereign country,380,IT,ITA,-99,-99,Feature
2,"[[[[-155.54211, 19.08348], [-155.68817, 18.916...",MultiPolygon,U.S.A.,6,USA,USA,-99,USA,-99,1,...,Northern America,United States of America,-99,Country,840,US,USA,0,-99,Feature
3,"[[[[-5.661948614921897, 54.55460317648385], [-...",MultiPolygon,U.K.,4,GBR,GBR,-99,GBR,-99,1,...,Northern Europe,United Kingdom,-99,Country,826,GB,GBR,-99,-99,Feature


### But the trick is how to merge these!

There is a whole lot of stuff in that Geo Jason file we just imported, but all we need is the geometry, and to find a column that we can merge with our nice simple output properties.

If you read through the columns, you will see there are many different formats for country names. But which one will match the country names that we have from BBC. 

`properties.subunit`

Looks the most promising for merging.

In [59]:
out_geometry['properties.subunit']

0                      Mexico
1                       Italy
2    United States of America
3              United Kingdom
Name: properties.subunit, dtype: object

### We DO need to change some names

So that we can match US and UK to our original data frame. The code below make sure that the countries in the column `properties.subunit` will match our columns.

In [60]:
out_geometry['properties.subunit'] = out_geometry['properties.subunit'].str.replace('United States of America', 'US')
out_geometry['properties.subunit'] = out_geometry['properties.subunit'].str.replace('United Kingdom', 'UK')
out_geometry

,geometry.coordinates,geometry.type,properties.abbrev,properties.abbrev_len,properties.adm0_a3,properties.adm0_a3_is,properties.adm0_a3_un,properties.adm0_a3_us,properties.adm0_a3_wb,properties.adm0_dif,...,properties.subregion,properties.subunit,properties.tiny,properties.type,properties.un_a3,properties.wb_a2,properties.wb_a3,properties.wikipedia,properties.woe_id,type
0,"[[[-97.14000830767071, 25.869997463478395], [-...",Polygon,Mex.,4,MEX,MEX,-99,MEX,-99,0,...,Central America,Mexico,-99,Sovereign country,484,MX,MEX,-99,-99,Feature
1,"[[[[15.520376010813834, 38.23115509699147], [1...",MultiPolygon,Italy,5,ITA,ITA,-99,ITA,-99,0,...,Southern Europe,Italy,-99,Sovereign country,380,IT,ITA,-99,-99,Feature
2,"[[[[-155.54211, 19.08348], [-155.68817, 18.916...",MultiPolygon,U.S.A.,6,USA,USA,-99,USA,-99,1,...,Northern America,US,-99,Country,840,US,USA,0,-99,Feature
3,"[[[[-5.661948614921897, 54.55460317648385], [-...",MultiPolygon,U.K.,4,GBR,GBR,-99,GBR,-99,1,...,Northern Europe,UK,-99,Country,826,GB,GBR,-99,-99,Feature


### Now it's time to merge!

We don't need all of those unnecessary properties from the geojson file we got. We just need the two geometry properties: 

'geometry.coordinates' and 'geometry.type'

Along with the column we are joining on (don't forget that one!): 'properties.subunit' 

So we are going to merge the `out_geometry` dataframe onto our nice, clean `out_properties` dataframe.

In [61]:
output_complete = pd.merge(out_properties, out_geometry[['geometry.coordinates','geometry.type','properties.subunit']], left_on='properties.name', right_on='properties.subunit', how='left')

In [62]:
output_complete

,properties.name,properties.article,properties.headline,properties.color,geometry.coordinates,geometry.type,properties.subunit
0,Italy,<p> At the First Breath of Wind : 1 vote</p><p...,1 critic,#000066,"[[[[15.520376010813834, 38.23115509699147], [1...",MultiPolygon,Italy
1,Mexico,"<p> 4 Months, 3 Weeks & 2 Days : 1 vote</p><p>...",1 critic,#000066,"[[[-97.14000830767071, 25.869997463478395], [-...",Polygon,Mexico
2,UK,"<p> 4 Months, 3 Weeks & 2 Days : 1 vote</p><p>...",1 critic,#000066,"[[[[-5.661948614921897, 54.55460317648385], [-...",MultiPolygon,UK
3,US,<p> Certified Copy : 2 votes</p><p> Eternal S...,4 critic,#000066,"[[[[-155.54211, 19.08348], [-155.68817, 18.916...",MultiPolygon,US


### Output that to geojson--for the leaflet template
The code below, is directly from the leaflet tutorial, with one modification that puts in the variable name for you. It will output your geojson document. What you can bring into leaflet (then, and only then, do you start thinking about leaflet!)

In [63]:
#transform your pandas dataframed to json
ok_json = json.loads(output_complete.to_json(orient='records'))

In [64]:
#The function below turms that json file into geojson format

In [65]:
def process_to_geojson(file):
    geo_data = {"type": "FeatureCollection", "features":[]}
    for row in file:
        this_dict = {"type": "Feature", "properties":{}, "geometry": {}}
        for key, value in row.items():
            key_names = key.split('.')
            if key_names[0] == 'geometry':
                this_dict['geometry'][key_names[1]] = value
            if str(key_names[0]) == 'properties':
                this_dict['properties'][key_names[1]] = value
        geo_data['features'].append(this_dict)
    return geo_data

In [66]:
#Pass the file through function
geo_format = process_to_geojson(ok_json)

In [67]:
#This puts in your variable name first
#Then put in the geojson object into your file
with open('geo-data.js', 'w') as outfile:
    outfile.write("var infoData = ")
with open('geo-data.js', 'a') as outfile:
    json.dump(geo_format, outfile)